In [ ]:
!pip uninstall -y torch
!pip uninstall -y torchtext
!pip uninstall -y torchvision

Uninstalling torch-1.9.0+cu102:
  Successfully uninstalled torch-1.9.0+cu102
Uninstalling torchtext-0.10.0:
  Successfully uninstalled torchtext-0.10.0
Uninstalling torchvision-0.10.0+cu102:
  Successfully uninstalled torchvision-0.10.0+cu102


In [ ]:
!pip install -q torch==1.7
!python -V
!pip install -q torchtext==0.8
!pip install -q torchvision==0.8

     |████████████████████████████████| 776.8MB 24kB/s 
ERROR: fastai 1.0.61 requires torchvision, which is not installed.
Python 3.7.10
     |████████████████████████████████| 7.0MB 5.2MB/s 
     |████████████████████████████████| 11.8MB 4.0MB/s 


In [ ]:
from torchsummary import summary
import numpy as np
import pandas as pd
import torch
import random
import spacy
import torch.nn as nn
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator
# from torchtext.legacy.data import Field, BucketIterator
from torchtext.data.metrics import bleu_score
import sys
import torchtext
import torchvision
print(torch.__version__)
print(torchtext.__version__)
print(torchvision.__version__)
torch.cuda.is_available()

1.7.0
0.8.0
0.8.0


True

In [ ]:
#Helper functions
def translate_sentence(model, sentence, german, english, device, max_length=50):
    # Load german tokenizer
    spacy_ger = spacy.load('de')

    # Create tokens using spacy and everything in lower case (which is what our vocab is)
    if type(sentence) == str:
        tokens = [token.text.lower() for token in spacy_ger(sentence)]
    else:
        tokens = [token.lower() for token in sentence]

    # Add <SOS> and <EOS> in beginning and end respectively
    tokens.insert(0, german.init_token)
    tokens.append(german.eos_token)

    # Go through each german token and convert to an index
    text_to_indices = [german.vocab.stoi[token] for token in tokens]

    # Convert to Tensor
    sentence_tensor = torch.LongTensor(text_to_indices).unsqueeze(1).to(device)

    outputs = [english.vocab.stoi["<sos>"]]
    for i in range(max_length):
        trg_tensor = torch.LongTensor(outputs).unsqueeze(1).to(device)

        with torch.no_grad():
            output = model(sentence_tensor, trg_tensor)

        best_guess = output.argmax(2)[-1, :].item()
        outputs.append(best_guess)

        if best_guess == english.vocab.stoi["<eos>"]:
            break

    translated_sentence = [english.vocab.itos[idx] for idx in outputs]
    # remove start token
    return translated_sentence[1:]


def bleu(data, model, german, english, device):
    targets = []
    outputs = []

    for example in data:
        src = vars(example)["src"]
        trg = vars(example)["trg"]

        prediction = translate_sentence(model, src, german, english, device)
        prediction = prediction[:-1]  # remove <eos> token

        targets.append([trg])
        outputs.append(prediction)

    return bleu_score(outputs, targets)


def save_checkpoint(state, filename="ger_to_engmulti30k.pth.tar"):
    print("=> Saving checkpoint")
    torch.save(state, filename)


def load_checkpoint(checkpoint, model, optimizer):
    print("=> Loading checkpoint")
    model.load_state_dict(checkpoint["state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer"])

In [ ]:
# For spacy 3.0
!python -m spacy download en
!python -m spacy download de

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')
     |████████████████████████████████| 14.9MB 4.4MB/s 
  Created wheel for de-core-news-sm: filename=de_core_news_sm-2.2.5-cp37-none-any.whl size=14907055 sha256=2093717eee6da7a077fd04074757257e5353047fc29403790fbd68ff41afc0b5
  Stored in directory: /tmp/pip-ephem-wheel-cache-waqbhctx/wheels/ba/3f/ed/d4aa8e45e7191b7f32db4bfad565e7da1edbf05c916ca7a1ca
Successfully built de-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/de_core_news_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/de
You can now load the model via spacy.load('de')


In [ ]:
spacy_ger = spacy.load('de')
spacy_eng = spacy.load('en')

In [ ]:
def tokenize_ger(text):
    return [tok.text for tok in spacy_ger.tokenizer(text)]
def tokenize_eng(text):
    return [tok.text for tok in spacy_eng.tokenizer(text)]

In [ ]:
german = Field(tokenize = tokenize_ger,lower = True, init_token = "<sos>",eos_token = "<eos>")
english = Field(tokenize = tokenize_eng,lower = True, init_token = "<sos>",eos_token = "<eos>")
train_data, valid_data, test_data = Multi30k.splits(exts=(".de", ".en"), fields=(german, english))
german.build_vocab(train_data, max_size=10000, min_freq=2)
english.build_vocab(train_data, max_size=10000, min_freq=2)

/usr/local/lib/python3.7/dist-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


downloading training.tar.gz


training.tar.gz: 100%|██████████| 1.21M/1.21M [00:02<00:00, 530kB/s]


downloading validation.tar.gz


validation.tar.gz: 100%|██████████| 46.3k/46.3k [00:00<00:00, 90.9kB/s]


downloading mmt_task1_test2016.tar.gz


mmt_task1_test2016.tar.gz: 100%|██████████| 66.2k/66.2k [00:00<00:00, 85.7kB/s]
/usr/local/lib/python3.7/dist-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)


In [ ]:
type(train_data)

torchtext.datasets.translation.Multi30k

In [ ]:
for i in range(5):
    print(' '.join(train_data[i].src), '--->', ' '.join(train_data[i].trg))

zwei junge weiße männer sind im freien in der nähe vieler büsche . ---> two young , white males are outside near many bushes .
mehrere männer mit schutzhelmen bedienen ein antriebsradsystem . ---> several men in hard hats are operating a giant pulley system .
ein kleines mädchen klettert in ein spielhaus aus holz . ---> a little girl climbing into a wooden playhouse .
ein mann in einem blauen hemd steht auf einer leiter und putzt ein fenster . ---> a man in a blue shirt is standing on a ladder cleaning a window .
zwei männer stehen am herd und bereiten essen zu . ---> two men are at the stove preparing food .


In [ ]:
for i in range(5):
    print(' '.join(valid_data[i].src), '--->', ' '.join(valid_data[i].trg))

eine gruppe von männern lädt baumwolle auf einen lastwagen ---> a group of men are loading cotton onto a truck
ein mann schläft in einem grünen raum auf einem sofa . ---> a man sleeping in a green room on a couch .
ein junge mit kopfhörern sitzt auf den schultern einer frau . ---> a boy wearing headphones sits on a woman 's shoulders .
zwei männer bauen eine blaue eisfischerhütte auf einem zugefrorenen see auf ---> two men setting up a blue ice fishing hut on an iced over lake
ein mann mit beginnender glatze , der eine rote rettungsweste trägt , sitzt in einem kleinen boot . ---> a balding man wearing a red life jacket is sitting in a small boat .


In [ ]:
print(f'The german vocabulary size is {len(german.vocab)}')
print(f'The english vocabulary size is {len(english.vocab)}')

The german vocabulary size is 7855
The english vocabulary size is 5893


In [ ]:
class Transformer(nn.Module):
    def __init__(
        self,
        embedding_size,
        src_vocab_size,
        trg_vocab_size,
        src_pad_idx,
        num_heads,
        num_encoder_layers,
        num_decoder_layers,
        forward_expansion,
        dropout,
        max_len,
        device,
    ):
        super(Transformer, self).__init__()
        self.src_word_embedding = nn.Embedding(src_vocab_size, embedding_size)
        self.src_position_embedding = nn.Embedding(max_len, embedding_size)
        self.trg_word_embedding = nn.Embedding(trg_vocab_size, embedding_size)
        self.trg_position_embedding = nn.Embedding(max_len, embedding_size)

        self.device = device
        self.transformer = nn.Transformer(
            embedding_size,
            num_heads,
            num_encoder_layers,
            num_decoder_layers,
            forward_expansion,
            dropout,
        )
        self.fc_out = nn.Linear(embedding_size, trg_vocab_size)
        self.dropout = nn.Dropout(dropout)
        self.src_pad_idx = src_pad_idx

    def make_src_mask(self, src):
        src_mask = src.transpose(0, 1) == self.src_pad_idx

        # (N, src_len)
        return src_mask.to(self.device)

    def forward(self, src, trg):
        src_seq_length, N = src.shape
        trg_seq_length, N = trg.shape

        src_positions = (
            torch.arange(0, src_seq_length)
            .unsqueeze(1)
            .expand(src_seq_length, N)
            .to(self.device)
        )

        trg_positions = (
            torch.arange(0, trg_seq_length)
            .unsqueeze(1)
            .expand(trg_seq_length, N)
            .to(self.device)
        )

        embed_src = self.dropout(
            (self.src_word_embedding(src) + self.src_position_embedding(src_positions))
        )
        embed_trg = self.dropout(
            (self.trg_word_embedding(trg) + self.trg_position_embedding(trg_positions))
        )

        src_padding_mask = self.make_src_mask(src)
        trg_mask = self.transformer.generate_square_subsequent_mask(trg_seq_length).to(
            self.device
        )

        out = self.transformer(
            embed_src,
            embed_trg,
            src_key_padding_mask=src_padding_mask,
            tgt_mask=trg_mask,
        )
        out = self.fc_out(out)
        return out

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
load_model = False
save_model = True
num_epochs = 300
learning_rate = 3e-5
batch_size = 32
ger_vocab_size = len(german.vocab)
eng_vocab_size = len(english.vocab)
embedding_size = 512
num_heads = 8
num_encoder_layers = 3
num_decoder_layers = 3
dropout = 0.10
max_len = 100
forward_expansion = 4
ger_pad_index = english.vocab.stoi["<pad>"]
writer = SummaryWriter("runs/loss_plot")
step = 0
print(ger_vocab_size)

In [ ]:
train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size=batch_size,
    sort_within_batch=True,
    sort_key=lambda x: len(x.src),
    device=device,
)

/usr/local/lib/python3.7/dist-packages/torchtext/data/iterator.py:48: UserWarning: BucketIterator class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [ ]:
model = Transformer(
    embedding_size,
    ger_vocab_size,
    eng_vocab_size,
    ger_pad_index,
    num_heads,
    num_encoder_layers,
    num_decoder_layers,
    forward_expansion,
    dropout,
    max_len,
    device,
).to(device)
optimizer = optim.Adam(model.parameters(), lr = learning_rate)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimiser, 
                                                       factor=0.1, 
                                                       patience=10, 
                                                       verbose=True)
pad_idx = english.vocab.stoi["<pad>"]
criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)
if load_model:
    load_checkpoint(torch.load("my_checkpoint.pth.tar"), model, optimizer)
sentence = "ein junge mit kopfhörern sitzt auf den schultern einer frau ."
# Translation:- a boy with headphones sits on the shoulders of a woman.

In [ ]:
for epoch in range(num_epochs):
    print(f"[Epoch {epoch + 1} / {num_epochs}]")

    if save_model:
        checkpoint = {
            "state_dict": model.state_dict(),
            "optimizer": optimizer.state_dict(),
        }
        save_checkpoint(checkpoint)
    # Evaluating on the test sentence defined above
    if epoch>=1:
        model.eval()
        translated_sentence = translate_sentence(
            model, sentence, german, english, device, max_length=100
        ) 
        print(f"Translated example sentence: \n {translated_sentence}")
    model.train()
    losses = []
    for batch_idx, batch in enumerate(train_iterator):
        # Get input and targets and get to cuda
        inp_data = batch.src.to(device)
        target = batch.trg.to(device)

        # Forward prop
        output = model(inp_data, target[:-1, :])
        output = output.reshape(-1, output.shape[2])
        target = target[1:].reshape(-1)

        optimizer.zero_grad()

        loss = criterion(output, target)
        losses.append(loss.item())

        # Back prop
        loss.backward()
        # Clip to avoid exploding gradient issues, makes sure grads are
        # within a healthy range
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)

        # Gradient descent step
        optimizer.step()

        # plot to tensorboard
        writer.add_scalar("Training loss", loss, global_step=step)
        step += 1

    mean_loss = sum(losses) / len(losses)
    scheduler.step(mean_loss)

[Epoch 1 / 300]
=> Saving checkpoint


/usr/local/lib/python3.7/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


[Epoch 2 / 300]
=> Saving checkpoint
Translated example sentence: 
 ['a', 'boy', 'with', 'a', 'woman', 'sitting', 'on', 'a', 'woman', 'in', 'the', 'grass', '.', '<eos>']
[Epoch 3 / 300]
=> Saving checkpoint
Translated example sentence: 
 ['a', 'boy', 'with', 'a', 'black', 'shirt', 'is', 'sitting', 'on', 'a', 'woman', '.', '<eos>']
[Epoch 4 / 300]
=> Saving checkpoint
Translated example sentence: 
 ['a', 'boy', 'with', 'a', 'hat', 'sitting', 'on', 'a', 'woman', '.', '<eos>']
[Epoch 5 / 300]
=> Saving checkpoint
Translated example sentence: 
 ['a', 'boy', 'with', 'a', 'red', 'hat', 'sitting', 'on', 'a', 'woman', '.', '<eos>']
[Epoch 6 / 300]
=> Saving checkpoint
Translated example sentence: 
 ['a', 'boy', 'with', 'a', 'red', 'hat', 'is', 'sitting', 'on', 'a', 'woman', '.', '<eos>']
[Epoch 7 / 300]
=> Saving checkpoint
Translated example sentence: 
 ['a', 'boy', 'with', 'a', 'red', 'hat', 'is', 'sitting', 'on', 'a', 'woman', '.', '<eos>']
[Epoch 8 / 300]
=> Saving checkpoint
Translated ex

In [ ]:
score = bleu(test_data[1:100], model, german, english, device)
print(f"Bleu score {score * 100:.2f}")

Bleu score 29.42


In [ ]:
batch = next(iter(train_iterator))
yhat = model(batch.src,batch.trg)

/usr/local/lib/python3.7/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [ ]:
!pip install torchviz

  Created wheel for torchviz: filename=torchviz-0.0.2-cp37-none-any.whl size=4152 sha256=d0e941a4e9c073469081aa7b8de714576f6757bc5b868893ab14e775e141b1e6
  Stored in directory: /root/.cache/pip/wheels/01/26/58/026ffd533dbe8b3972eb423da9c7949beca68d1c98ed9e8624
Successfully built torchviz


In [ ]:
from torchviz import make_dot
make_dot(yhat, params=dict(list(model.named_parameters()))).render("Ger_to_Eng_Transformer", format="png")

'Ger_to_Eng_Transformer.png'

In [ ]:
batch.src.shape
batch.trg.shape

torch.Size([22, 32])

In [ ]:
print(model)

Transformer(
  (src_word_embedding): Embedding(7855, 512)
  (src_position_embedding): Embedding(100, 512)
  (trg_word_embedding): Embedding(5893, 512)
  (trg_position_embedding): Embedding(100, 512)
  (transformer): Transformer(
    (encoder): TransformerEncoder(
      (layers): ModuleList(
        (0): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): _LinearWithBias(in_features=512, out_features=512, bias=True)
          )
          (linear1): Linear(in_features=512, out_features=4, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_features=4, out_features=512, bias=True)
          (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(p=0.1, inplace=False)
          (dropout2): Dropout(p=0.1, inplace=False)
        )
        (1): TransformerEncoderLayer(
          (self_attn): MultiheadAtte